## Import dependencies 

In [1]:
## Set path context
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)
print(sys.path)

['/workspaces/genai-ws/genai_workshop', '/workspaces/genai-ws/genai_workshop/example_notebook', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [2]:
# --------------------------------------------------------------
# Import Modules
# --------------------------------------------------------------

from prompt_utils.openai_utils import get_openai_chat_model
import os
import json
from datetime import datetime, timedelta
from openai import OpenAI


### Open AI function call 

In [3]:

user_prompt = """
    When's the next flight from Amsterdam to New York?"
"""


# --------------------------------------------------------------
# Ask ChatGPT a Question
# --------------------------------------------------------------

completion = OpenAI().chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
)

output = completion.choices[0].message.content
print(output)


I'm sorry, but as an AI language model, I don't have access to real-time information. It's best to check with the airlines or a travel website for the most up-to-date flight schedules.


In [4]:
# --------------------------------------------------------------
# Use OpenAI’s Function Calling Feature
# --------------------------------------------------------------

function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations.",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    },
    
]


completion = OpenAI().chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)


# It automatically fills the arguments with correct info based on the prompt
# Note: the function does not exist yet

output = completion.choices[0].message
print(output)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', name='get_flight_info'), tool_calls=None)


In [5]:

# --------------------------------------------------------------
# Add a Function
# --------------------------------------------------------------


def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)


# Use the LLM output to manually call the function
# The json.loads function converts the string to a Python object

origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)
type(params)

print(origin)
print(destination)
print(params)


AMS
JFK
{'loc_origin': 'AMS', 'loc_destination': 'JFK'}


In [6]:

# Call the function second time

chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)

print(flight)


{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-02-12 04:24:36.104475", "airline": "KLM", "flight": "KL643"}


In [7]:
# --------------------------------------------------------------
# Add function result to the prompt for a final answer
# --------------------------------------------------------------

# The key is to add the function output back to the messages with role: function
second_completion = OpenAI().chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)

The next flight from Amsterdam (AMS) to New York (JFK) is on February 12, 2024. The flight is operated by KLM (airline) and the flight number is KL643.
